# Qwen 2.5

```{note}
Compared to previous iterations, Qwen 2.5 has
been significantly improved during both the pre-training and post-training stages. In
terms of pre-training, we have scaled the high-quality pre-training datasets from the
previous 7 trillion tokens to 18 trillion tokens. In terms of post-training,
we implement intricate supervised finetuning with over 1 million samples, as well as
multistage reinforcement learning, including `offline learning DPO` and `online learning
GRPO`.<br>
To handle diverse and varied use cases effectively, we present Qwen2.5 LLM series in rich
configurations. Specifically, the open-weight flagship Qwen2.5-72B-Instruct outperforms
a number of open and proprietary models and demonstrates competitive performance to
the state-of-the-art open-weight model, Llama-3-405B-Instruct, which is around 5 times
larger.
```

## Architecture & Tokenizer

Basically, the Qwen2.5 series include dense models for opensource, and MoE models for API service.

For dense models, we maintain the Transformer-based decoder architecture as Qwen2. The architecture incorporates several key components:
Grouped Query Attention (GQA) for efficient KV cache utilization, SwiGLU activation
function for non-linear activation, Rotary Positional Embeddings (RoPE) for encoding position information, QKV bias in the attention mechanism and
RMSNorm with pre-normalization to ensure stable training.

Building upon the dense model architectures, we extend it to MoE model architectures. This is achieved
by replacing standard feed-forward network (FFN) layers with specialized MoE layers.

For tokenization, we utilize Qwen’s tokenizer, which implements byte-level byte-pair
encoding (BBPE) with a vocabulary of 151,643
regular tokens.

## Pre-training

### Pre-training Data

Qwen2.5 demonstrates significant enhancements in pre-training data quality compared to its predecessor
Qwen2. These improvements stem from several key aspects:

1. Better data filtering.
2. Better math and code data.
3. Better synthetic data.
4. Better data mixture.

### Scaling Law for Hyper-parameters

We develop scaling laws for hyper-parameter based on the pre-training data of Qwen2.5. While previous studies primarily used scaling laws to determine optimal model sizes given compute budgets, we
leverage them to identify optimal hyperparameters across model architectures.

## Post-training

Qwen 2.5 introduces two significant advancements in its post-training design compared to Qwen 2:

1. Expanded Supervised Fine-tuning Data Coverage: The supervised fine-tuning process leverages
a massive dataset comprising millions of high-quality examples.
2. Two-stage Reinforcement Learning: The reinforcement learning (RL) process in Qwen 2.5 is
divided into two distinct stages: Offline RL and Online RL.
    * Offline RL: This stage focuses on developing capabilities that are challenging for the reward
model to evaluate, such as reasoning, factuality, and instruction-following.
    * Online RL: The Online RL phase leverages the reward model’s ability to detect nuances in
output quality.

### Supervised Fine-tuning

In this section, we detail the key enhancements made during the SFT phase of Qwen2.5, focusing on
several critical areas:

**Coding:** To enhance coding capabilities, we incorporate the instruction tuning data of Qwen2.5-
Coder. We use multiple language-specific agents into a collaborative framework,
generating diverse and high-quality instruction pairs across nearly 40 programming languages.
We expand our instruction dataset by `synthesizing new examples from code-related Q&A
websites` and gathering `algorithmic code snippets from GitHub`. A `comprehensive multilingual
sandbox` is used to perform static code checking and validate code snippets through automated
unit testing, ensuring code quality and correctness.

### Offline Reinforcement Learning

Compared to Online Reinforcement Learning (RL), Offline RL enables the pre-preparation of training
signals, which is particularly advantageous for tasks where standard answers exist but are challenging to
evaluate using reward model. In this study, we focus on objective query domains such as mathematics,
coding, instruction following, and logical reasoning, where obtaining accurate evaluations can be complex.
In the previous phase, we extensively employ strategies like `execution feedback` and answer matching to
ensure the quality of responses. For the current phase, we reuse that pipeline, employing the SFT model
to resample responses for a new set of queries. Responses that pass our quality checks are used as positive
examples, while those that fail are treated as negative examples for Direct Preference Optimization (DPO). To further enhance the reliability and accuracy of the training signals, we
make use of both `human and automated review` processes.

### Online Reinforcement Learning

To develop a robust reward model for online RL, we adhere to a set of carefully defined labeling criteria:

* Truthfulness
* Helpfulness
* Conciseness
* Relevance
* Harmlessness
* Debiasing

The queries utilized to train the reward model are drawn from two distinct datasets: `publicly available
open-source data` and a `proprietary query set characterized by higher complexity`. Responses are generated
from checkpoints of the Qwen models, which have been fine-tuned using different methods—SFT,
DPO, and RL—at various stages of training. To introduce diversity, those responses are sampled at
`different temperature settings`. Preference pairs are created through both human and automated labeling
processes, and the training data for DPO is also integrated into this dataset.

In our online reinforcement learning (RL) framework, we employ Group Relative Policy Optimization. `The query set utilized for training the reward model is identical to the one used
in the RL training phase`. The sequence in which queries are processed during training is determined by
the variance of their response scores, as evaluated by the reward model. Specifically, queries with higher
variance in response scores are prioritized to ensure more effective learning. We sample 8 responses
for each query.

## Evaluation

### Base Models

```{figure} ../images/qwen25-1.png
```

### Instruction-tuned Model

```{figure} ../images/qwen25-2.png
```